In [110]:
class Transition:
    def __init__(self, ins, neigh, outs, symm='none'):
        self.ins = ins
        self.outs = outs
        #NESW
        self.neigh = neigh
        self.symm = symm
        assert len(neigh)==4

    def to_str(self, outtransform, outstate):
        if (outstate[0]==self.symm):
            o=''
        else:
            o = f'symmetries:{self.symm}\n'
            outstate[0]=self.symm
        o = o + self.ins.textrepr()
        for c in self.neigh:
            o = o+ ", " + c.textrepr()
        o = o+"; "+str(outtransform(self.outs))
        return o

class NamedGroup(str):
    def textrepr(self):
        return self

    

In [111]:
def create_colors(statecolors, devirtualize):
    colorsstr = ''
    for s,c in statecolors.items():
        colorsstr += f'{int(255*c[0])} {int(255*c[1])} {int(255*c[2])} : {devirtualize(s)}\n'
    return colorsstr

def create_icons(stateicons, devirtualize):
    icosstr = ''
    for s,ic in stateicons.items():
        icosstr += f'#C {devirtualize(s)}\nx=31, y=31, rule=//10\n{ic.strip()}\n\n'
    return icosstr


In [112]:
def doperm(s, perm):
    for chain in perm:
        if s in chain:
            return chain[(chain.index(s)+1)%len(chain)]
    return s

def create_table(groups, trans, devirtualize, perm):
    outpstr = ''
    for n,g in groups.items():
        outpstr += f'{n} = {tuple(map(devirtualize,g))}\n'
    os = ['None', ]
    for t in trans:
        outpstr += t.to_str(lambda s: devirtualize(doperm(s, perm)),os) + '\n'
    return outpstr

In [113]:
from dataclasses import dataclass
import itertools



@dataclass(frozen=True)
class FSGlass:
    active : bool
    state : int

    def textrepr(self):
        return str(devirtualize(self))
    
@dataclass(frozen=True)
class FSGlass:
    active : bool
    state : int

    def textrepr(self):
        return str(devirtualize(self))




groups = dict(
    active = [FSGlass(True, i) for i in range(3)],
    passive = [FSGlass(False, i) for i in range(3)],
)


trans = []
for c,n,e,s,w in itertools.product(*([[FSGlass(True,i) for i in range(3)], ] + 4*[[FSGlass(False,i) for i in range(3)], ])):
    t = lambda x : 0 if x.state==2 else 2*x.state-1
    if c.state!=2 and -t(n)+t(e)+t(s)+t(w)==0:
        trans.append(Transition(c, [n,e,s,w], FSGlass(True, 1-c.state)))

for i in range(3):
    trans.append(Transition(FSGlass(False, i), [NamedGroup('active')]*4, FSGlass(False, i)))
    trans.append(Transition(FSGlass(True, i), [NamedGroup('passive')]*4, FSGlass(True, i)))

perm = []
for i in range(3):
    perm.append([FSGlass(True, i),FSGlass(False, i)])


def devirtualize(s):
    if isinstance(s, FSGlass):
        return 1 + s.active*3 + s.state
    elif s==0:
        return 0
    else:
        raise "wtf"
    
statecolors = {}
for i in range(3):
    statecolors[FSGlass(True,i)] = (0,i*0.5,1)
    statecolors[FSGlass(False,i)] = (0,i*0.45,0.9)

finalfile = ('''
@NUTSHELL testing_temp

@TABLE
neighborhood: vonNeumann
[[[PASTE TRANSITIONS HERE]]]
any, any, any, any, any; 0

@COLORS
0 0 0 : 0
[[[PASTE COLORS HERE]]]

'''.replace('[[[PASTE TRANSITIONS HERE]]]', create_table(groups,trans,devirtualize,perm))
   .replace('[[[PASTE COLORS HERE]]]', create_colors(statecolors, devirtualize))
)

with open('testing_temp.ruel','w') as fh:
    fh.write(finalfile)



In [114]:
from dataclasses import dataclass
import itertools



@dataclass(frozen=True)
class FSGlass:
    active : bool
    state : int

    def textrepr(self):
        return str(devirtualize(self))
    

@dataclass(frozen=True)
class FSGBoundary:
    state : FSGlass

    def textrepr(self):
        return str(devirtualize(self))

@dataclass(frozen=True)
class Ising:
    active : bool
    state : int

    def textrepr(self):
        return str(devirtualize(self))

@dataclass(frozen=True)
class IBoundary:
    state : Ising

    def textrepr(self):
        return str(devirtualize(self))

#parity locked
@dataclass(frozen=True)
class FSGI:
    state1 : Ising
    state2 : FSGlass
    ising_e : int
    fsg_e : int

    def textrepr(self):
        return str(devirtualize(self))

all_ising = [Ising(True,0),Ising(False,0),Ising(True,1),Ising(False,1)]
all_fsg = [FSGlass(True,0),FSGlass(False,0),FSGlass(True,1),FSGlass(False,1)]



groups = dict(
    active_fsg = [FSGlass(True, i) for i in range(2)] + [FSGBoundary(FSGlass(True, i)) for i in range(2)] + [FSGI(Ising(True, i),FSGlass(True, j),0,0) for i,j in itertools.product(range(2), range(2))],
    passive_fsg = [FSGlass(False, i) for i in range(2)] + [FSGBoundary(FSGlass(False, i)) for i in range(2)] + [FSGI(Ising(False, i),FSGlass(False, j),0,0) for i,j in itertools.product(range(2), range(2))],
    up_p_fsg = [FSGlass(False,1), FSGBoundary(FSGlass(False,1))] + [FSGI(Ising(False, i),FSGlass(False, 1),0,0) for i in range(2)],
    down_p_fsg = [FSGlass(False,0), FSGBoundary(FSGlass(False,0))] + [FSGI(Ising(False, i),FSGlass(False, 1),0,0) for i in range(2)],

    active_i = [Ising(True, i) for i in range(2)] + [IBoundary(Ising(True, i)) for i in range(2)] + [FSGI(Ising(True, i),FSGlass(True, j),0,0) for i,j in itertools.product(range(2), range(2))],
    passive_i = [Ising(False, i) for i in range(2)] + [IBoundary(Ising(False, i)) for i in range(2)] + [FSGI(Ising(False, i),FSGlass(False, j),0,0) for i,j in itertools.product(range(2), range(2))],
    up_p_i = [Ising(False,1), IBoundary(Ising(False,1))] + [FSGI(Ising(False, 1),FSGlass(False, i),0,0) for i in range(2)],
    down_p_i = [Ising(False,0), IBoundary(Ising(False,0))] + [FSGI(Ising(False, 0),FSGlass(False, i),0,0) for i in range(2)],

    active_fsgi = [FSGI(Ising(True, i),FSGlass(True, j),0,0) for i,j in itertools.product(range(2), range(2))],
    passive_fsgi = [FSGI(Ising(False, i),FSGlass(False, j),0,0) for i,j in itertools.product(range(2), range(2))],
)

trans = []
for c,n,e,s,w in itertools.product([FSGlass(True,0), FSGlass(True,1)], *(

    4*[[NamedGroup('up_p_fsg'),NamedGroup('down_p_fsg')],]
    
    )):

    t = lambda x : 1 if str(x)=='up_p_fsg' else -1
    if t(n)-t(e)+t(s)+t(w)==0:
        trans.append(Transition(c, [n,e,s,w], FSGlass(True, 1-c.state))) # type: ignore

for i in range(2):
    trans.append(Transition(FSGlass(False, i), [NamedGroup('active_fsg')]*4, FSGlass(False, i)))
    trans.append(Transition(FSGlass(True, i), [NamedGroup('passive_fsg')]*4, FSGlass(True, i)))

for i in range(2):
    trans.append(Transition(FSGBoundary(FSGlass(False, i)), [NamedGroup('(active_fsg,0)')]*4, FSGBoundary(FSGlass(False, i))))
    trans.append(Transition(FSGBoundary(FSGlass(True, i)), [NamedGroup('(passive_fsg,0)')]*4, FSGBoundary(FSGlass(True, i))))


for c,n,e,s,w in itertools.product([Ising(True,0), Ising(True,1)], *(

    4*[[NamedGroup('up_p_i'),NamedGroup('down_p_i')],]
    
    )):

    t = lambda x : 1 if str(x)=='up_p_i' else -1
    if t(n)+t(e)+t(s)+t(w)==0:
        trans.append(Transition(c, [n,e,s,w], Ising(True, 1-c.state))) # type: ignore

for i in range(2):
    trans.append(Transition(Ising(False, i), [NamedGroup('active_i')]*4, Ising(False, i)))
    trans.append(Transition(Ising(True, i), [NamedGroup('passive_i')]*4, Ising(True, i)))

for i in range(2):
    trans.append(Transition(IBoundary(Ising(False, i)), [NamedGroup('(active_i,0)')]*4, IBoundary(Ising(False, i))))
    trans.append(Transition(IBoundary(Ising(True, i)), [NamedGroup('(passive_i,0)')]*4, IBoundary(Ising(True, i))))


for i in range(2):
    for j in range(2):
        trans.append(Transition(
            FSGI(Ising(False, i), FSGlass(False,j),0,0), 
            [NamedGroup('(active_fsgi,0)'), NamedGroup('active_i'), NamedGroup('(active_fsgi,0)'), NamedGroup('active_fsg'),],
            FSGI(Ising(False, i), FSGlass(False,j),0,0)
        ))

        trans.append(Transition(
            FSGI(Ising(True, i), FSGlass(True,j),0,0), 
            [NamedGroup('(passive_fsgi,0)'), Ising(False, 1-i), NamedGroup('(passive_fsgi,0)'), FSGlass(False, 1-j),],
            FSGI(Ising(True, i), FSGlass(True,j),1,1)
        ))

        trans.append(Transition(
            FSGI(Ising(True, i), FSGlass(True,j),0,0), 
            [NamedGroup('(passive_fsgi,0)'), Ising(False, i), NamedGroup('(passive_fsgi,0)'), FSGlass(False, 1-j),],
            FSGI(Ising(True, i), FSGlass(True,j),-1,1)
        ))

        trans.append(Transition(
            FSGI(Ising(True, i), FSGlass(True,j),0,0), 
            [NamedGroup('(passive_fsgi,0)'), Ising(False, 1-i), NamedGroup('(passive_fsgi,0)'), FSGlass(False, j),],
            FSGI(Ising(True, i), FSGlass(True,j),1,-1)
        ))

        trans.append(Transition(
            FSGI(Ising(True, i), FSGlass(True,j),0,0), 
            [NamedGroup('(passive_fsgi,0)'), Ising(False, i), NamedGroup('(passive_fsgi,0)'), FSGlass(False, j),],
            FSGI(Ising(True, i), FSGlass(True,j),-1,-1)
        ))


perm = []
for i in range(2):
    perm.append([FSGlass(True, i),FSGlass(False, i)])
    perm.append([FSGBoundary(FSGlass(True, i)), FSGBoundary(FSGlass(False, i))])

    perm.append([Ising(True, i),Ising(False, i)])
    perm.append([IBoundary(Ising(True, i)), IBoundary(Ising(False, i))])

    for e1 in [-1,0,1]:
        for e2 in [-1,0,1]:
            perm.append([FSGI(Ising(True, i), FSGlass(True,i),e1,e2), FSGI(Ising(False, i), FSGlass(False,i),e1,e2)])



def devirtualize(s):
    if isinstance(s, FSGlass):
        return 1 + s.active*2 + s.state
    elif isinstance(s, FSGBoundary):
        return 5 + s.state.active*2 + s.state.state
    elif isinstance(s, Ising):
        return 9 + s.active*2 + s.state
    elif isinstance(s, IBoundary):
        return 13 + s.state.active*2 + s.state.state
    elif isinstance(s, FSGI):
        return 17 + s.state1.active*4 + s.state1.state*2 + + s.state2.state
    elif s==0:
        return 0
    else:
        raise Exception()
    
statecolors = {}
for i in range(2):
    statecolors[FSGlass(True,i)] = (0,i*0.5,0.6)
    statecolors[FSGlass(False,i)] = (0,i*0.45,0.5)
    statecolors[FSGBoundary(FSGlass(True,i))] = (0.5,i*0.5,0.6)
    statecolors[FSGBoundary(FSGlass(False,i))] = (0.5,i*0.45,0.5)

    statecolors[Ising(True,i)] = (0.1,i*0.5,0.6)
    statecolors[Ising(False,i)] = (0.1,i*0.45,0.5)
    statecolors[IBoundary(Ising(True,i))] = (0.6,i*0.5,0.6)
    statecolors[IBoundary(Ising(False,i))] = (0.6,i*0.45,0.5)


finalfile = ('''
@NUTSHELL testing_temp

@TABLE
neighborhood: vonNeumann
[[[PASTE TRANSITIONS HERE]]]
any, any, any, any, any; 0

@COLORS
0 0 0 : 0
[[[PASTE COLORS HERE]]]

'''.replace('[[[PASTE TRANSITIONS HERE]]]', create_table(groups,trans,devirtualize,perm))
   .replace('[[[PASTE COLORS HERE]]]', create_colors(statecolors, devirtualize))
)

with open('better_temp.ruel','w') as fh:
    fh.write(finalfile)
